In [1]:
#Lets start with SpartSession
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("PySpark DataFrame and Sql") \
    .getOrCreate()

In [2]:
# lets read the data given in assignment
NYC_df= spark.read.format("csv")\
 .option("header","true")\
 .option("inferSchema","true")\
 .load('/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv')\

In [3]:
# lets check the structure of the file
NYC_df.schema

StructType(List(StructField(Summons Number,LongType,true),StructField(Plate ID,StringType,true),StructField(Registration State,StringType,true),StructField(Issue Date,TimestampType,true),StructField(Violation Code,IntegerType,true),StructField(Vehicle Body Type,StringType,true),StructField(Vehicle Make,StringType,true),StructField(Violation Precinct,IntegerType,true),StructField(Issuer Precinct,IntegerType,true),StructField(Violation Time,StringType,true)))

In [4]:
# lets further check the structure of the file in more readable way
NYC_df.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Violation Time: string (nullable = true)



In [5]:
# lets view the file (first 20 rows)
NYC_df.show()

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|
|    4006265037| FZX9232|                NY|2016-08-23 00:00:00|             5|             SUBN|        FORD|                 0|              0|         0233P|
|    8478629828| 66623ME|         

In [6]:
# lets describe the dataframe once
NYC_df.describe().show()

+-------+-------------------+--------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+
|summary|     Summons Number|Plate ID|Registration State|    Violation Code| Vehicle Body Type|      Vehicle Make|Violation Precinct|  Issuer Precinct|   Violation Time|
+-------+-------------------+--------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+
|  count|           10803028|10803028|          10803028|          10803028|          10803028|          10803028|          10803028|         10803028|         10803028|
|   mean|6.817447029065661E9|Infinity|              99.0|34.599430455979565|3.9258887134586864| 6519.974025974026| 45.01216260848347|46.82931211508477|909.2857142857143|
| stddev|2.320233962328229E9|     NaN|               0.0|19.359868716323483|0.5013415469252523|18091.257389147086|40.552560268435805|62.66703577269466

In [7]:
# as above description is not very clear,lets describe one single field and check once
NYC_df.describe('Violation Code').show()

+-------+------------------+
|summary|    Violation Code|
+-------+------------------+
|  count|          10803028|
|   mean|34.599430455979565|
| stddev|19.359868716323483|
|    min|                 0|
|    max|                99|
+-------+------------------+



#### Please Note: As suggested in the expert sessions, here we are assuming that all the data is pertaining to 2017 as per file name

In [8]:
# lets check how many rows are there in file or should say total number of tickets
NYC_df.count()

10803028

In [9]:
# lets see if there are any rows having null 
NYC_df.dropna(how = 'any').count()  #total number of row count was also 10803028, which means there are no rows having null values

10803028

In [10]:
NYC_df.select('Issue Date').distinct().count()

2063

import pandas as pd
import matplotlib.pyplot as plt
plt.clf()
NYC_pdDF = NYC_df.toPandas()
NYC_pdDF.plot(x='Issue Date', y='Violation Code', kind='bar', rot=45)
plt.show()

## Examine the data

In [11]:
#1.Find the total number of tickets for the year
NYC_df.select('Registration State').distinct().count()

67

In [12]:
#2.a.Find out the number of unique states from where the cars that got parking tickets came.Hint: Use the column 'Registration State'.
from pyspark.sql.functions import col
NYC_df.cube('Registration State').count().sort(col("count").desc()).show()


+------------------+--------+
|Registration State|   count|
+------------------+--------+
|              null|10803028|
|                NY| 8481061|
|                NJ|  925965|
|                PA|  285419|
|                FL|  144556|
|                CT|  141088|
|                MA|   85547|
|                IN|   80749|
|                VA|   72626|
|                MD|   61800|
|                NC|   55806|
|                IL|   37329|
|                GA|   36852|
|                99|   36625|
|                TX|   36516|
|                AZ|   26426|
|                OH|   25302|
|                CA|   24260|
|                SC|   21836|
|                ME|   21574|
+------------------+--------+
only showing top 20 rows



In [13]:
#2.b.There is a numeric entry '99' in the column, which should be corrected. Replace it with the state having the maximum entries.
 
from pyspark.sql.functions import when

NYC_df1 = NYC_df.withColumn("Registration State", \
              when(NYC_df["Registration State"] == '99', 'NY').otherwise(NYC_df["Registration State"]))


In [14]:
#2.b.Provide the number of unique states again
NYC_df1.cube('Registration State').count().sort(col("count").desc()).show()  

+------------------+--------+
|Registration State|   count|
+------------------+--------+
|              null|10803028|
|                NY| 8517686|
|                NJ|  925965|
|                PA|  285419|
|                FL|  144556|
|                CT|  141088|
|                MA|   85547|
|                IN|   80749|
|                VA|   72626|
|                MD|   61800|
|                NC|   55806|
|                IL|   37329|
|                GA|   36852|
|                TX|   36516|
|                AZ|   26426|
|                OH|   25302|
|                CA|   24260|
|                SC|   21836|
|                ME|   21574|
|                MN|   18227|
+------------------+--------+
only showing top 20 rows



#### 8481061(NY)+36625(99) = 8517686(When NY replaced value 99)

## Aggregation tasks

In [15]:
#1.How often does each violation code occur? Display the frequency of the top five violation codes.
NYC_df.cube('Violation Code').count().sort(col("count").desc()).show(6)

+--------------+--------+
|Violation Code|   count|
+--------------+--------+
|          null|10803028|
|            21| 1528588|
|            36| 1400614|
|            38| 1062304|
|            14|  893498|
|            20|  618593|
+--------------+--------+
only showing top 6 rows



In [16]:
#2.a.How often does each 'vehicle body type' get a parking ticket? 
NYC_df.cube('Vehicle Body Type').count().sort(col("count").desc()).show(6)

+-----------------+--------+
|Vehicle Body Type|   count|
+-----------------+--------+
|             null|10803028|
|             SUBN| 3719802|
|             4DSD| 3082020|
|              VAN| 1411970|
|             DELV|  687330|
|              SDN|  438191|
+-----------------+--------+
only showing top 6 rows



In [17]:
#2.b.How about the 'vehicle make'? 
NYC_df.cube('Vehicle Make').count().sort(col("count").desc()).show(6)

+------------+--------+
|Vehicle Make|   count|
+------------+--------+
|        null|10803028|
|        FORD| 1280958|
|       TOYOT| 1211451|
|       HONDA| 1079238|
|       NISSA|  918590|
|       CHEVR|  714655|
+------------+--------+
only showing top 6 rows



In [18]:
#3.A precinct is a police station that has a certain zone of the city under its command. 
#Find the (5 highest) frequencies of tickets for each of the following:

#3.1.'Violation Precinct' (This is the precinct of the zone where the violation occurred). 
#Using this, can you draw any insights for parking violations in any specific areas of the city?
NYC_df.cube('Violation Precinct').count().sort(col("count").desc()).show(6)

+------------------+--------+
|Violation Precinct|   count|
+------------------+--------+
|              null|10803028|
|                 0| 2072400|
|                19|  535671|
|                14|  352450|
|                 1|  331810|
|                18|  306920|
+------------------+--------+
only showing top 6 rows



#### There is default value 0 with most of the vialotions, which doesn't seem to be right

In [19]:
#3.2.'Issuer Precinct' (This is the precinct that issued the ticket.)
#Here, you would have noticed that the dataframe has the'Violating Precinct' or 'Issuing Precinct' as '0'. 
#These are erroneous entries. Hence, you need to provide the records for five correct precincts. 
#(Hint: Print the top six entries after sorting.)
NYC_df.cube('Issuer Precinct').count().sort(col("count").desc()).show(6)

+---------------+--------+
|Issuer Precinct|   count|
+---------------+--------+
|           null|10803028|
|              0| 2388479|
|             19|  521513|
|             14|  344977|
|              1|  321170|
|             18|  296553|
+---------------+--------+
only showing top 6 rows



In [20]:
#4.Find the violation code frequencies for three precincts that have issued the most number of tickets. 
#Do these precinct zones have an exceptionally high frequency of certain violation codes? 
#Are these codes common across precincts? 
#(Hint: In the SQL view, use the 'where' attribute to filter among three precincts.)

#lets first creat a temp table

In [21]:
# register DataFrame as temp table
NYC_df.createOrReplaceTempView("NYC_dfTable")
spark.sql('SELECT * FROM NYC_dfTable').show(5)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|
|    4006265037| FZX9232|                NY|2016-08-23 00:00:00|             5|             SUBN|        FORD|                 0|              0|         0233P|
|    8478629828| 66623ME|         

In [22]:
#4.2 now lets fetch the maxinmum violation code counts for top three Issuer Precinct i.e. 19,14 and 1(as obtained from earlier tasks answer)
spark.sql('SELECT `Issuer Precinct`,`violation code`,count(`violation code`) FROM NYC_dfTable where `Issuer Precinct` in (19,14,1) group by `Issuer Precinct`,`violation code` order by count(`violation code`) desc ').show()

+---------------+--------------+---------------------+
|Issuer Precinct|violation code|count(violation code)|
+---------------+--------------+---------------------+
|             19|            46|                86390|
|             14|            14|                73837|
|              1|            14|                73522|
|             19|            37|                72437|
|             19|            38|                72344|
|             14|            69|                58026|
|             19|            14|                57563|
|             19|            21|                54700|
|             14|            31|                39857|
|              1|            16|                38937|
|             19|            16|                31353|
|             14|            47|                30540|
|              1|            20|                27841|
|             19|            20|                27352|
|              1|            46|                22534|
|         

#### Above query results show that maximum parking violations in Issuer Precinct# 19 are due to Double Parking(code# 46),followed by General No Standing(code# 14) in Issuer Precinct# 14 and 1

In [23]:
#5.Find out the properties of parking violations across different times of the day:

# lets see what are top 20 violations happenning as per violation time 
spark.sql('SELECT `Violation Time`,`violation code`,count(`violation code`) FROM NYC_dfTable group by `Violation Time`,`violation code` order by count(`violation code`) desc ').show()


+--------------+--------------+---------------------+
|Violation Time|violation code|count(violation code)|
+--------------+--------------+---------------------+
|         0836A|            21|                17213|
|         1136A|            21|                14143|
|         0840A|            21|                13140|
|         1140A|            21|                12917|
|         0806A|            21|                12525|
|         0838A|            21|                12266|
|         0837A|            21|                12023|
|         0839A|            21|                11962|
|         0936A|            21|                11747|
|         0841A|            21|                11363|
|         0842A|            21|                11060|
|         1138A|            21|                11041|
|         1139A|            21|                10917|
|         0845A|            21|                10858|
|         1145A|            21|                10752|
|         1142A|            

#### Above shows most of the violation happenning are for violation code#21(Street Cleaning)

In [24]:
#5.1.Find a way to deal with missing values, if any.
#(Hint: Check for the null values using 'isNull' under the SQL. Also, to remove the null values, check the 'dropna' command in the API documentation.)
spark.sql('SELECT * FROM NYC_dfTable where `Violation Time` is Null or `Violation Time` = "" ').show()

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+



#### As per above Violation Time doesn't seem to have any NULL or empty values

In [25]:
#5.2.The Violation Time field is specified in a strange format. Find a way to make this a time attribute that you can use to divide into groups.
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

NYC_df_HR = udf(lambda x:x[0:2],StringType())
NYC_df_MIN = udf(lambda x:x[2:4],StringType())
NYC_df_AMPM = udf(lambda x:x[4:5],StringType())
NYC_df = NYC_df.withColumn('Violation Hour',NYC_df_HR('Violation Time'))
NYC_df = NYC_df.withColumn('Violation Min',NYC_df_MIN('Violation Time'))
NYC_df = NYC_df.withColumn('Violation AMPM',NYC_df_AMPM('Violation Time'))
NYC_df.show()

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+--------------+-------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|Violation Hour|Violation Min|Violation AMPM|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+--------------+-------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|            01|           43|             A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|            04|     

In [26]:
##Updating the SQL object
NYC_df.createOrReplaceTempView("NYC_dfTable")
spark.sql('SELECT * FROM NYC_dfTable').show(5)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+--------------+-------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|Violation Hour|Violation Min|Violation AMPM|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+--------------+-------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|            01|           43|             A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|            04|     

In [27]:
#5.3 Divide 24 hours into six equal discrete bins of time. Choose the intervals as you see fit. For each of these groups, find the three most commonly occurring violations.
spark.sql('SELECT `violation code`,count(`violation code`), \
            case \
            when `Violation Hour` IN ("12","01","02","03") and `Violation AMPM` = "A" THEN "group12to03AM" \
            when `Violation Hour` IN ("04","05","06","07") and `Violation AMPM` = "A" THEN "group04to07AM" \
            when `Violation Hour` IN ("08","09","10","11") and `Violation AMPM` = "A" THEN "group08to11AM" \
            when `Violation Hour` IN ("12","01","02","03") and `Violation AMPM` = "P" THEN "group12to03PM" \
            when `Violation Hour` IN ("04","05","06","07") and `Violation AMPM` = "P" THEN "group04to07PM" \
            when `Violation Hour` IN ("08","09","10","11") and `Violation AMPM` = "P" THEN "group08to11PM" \
            END AS time_bin \
            FROM NYC_dfTable \
            group by `violation code`,time_bin\
            order by count(`violation code`) desc').show(100)

+--------------+---------------------+-------------+
|violation code|count(violation code)|     time_bin|
+--------------+---------------------+-------------+
|            21|              1182689|group08to11AM|
|            36|               751422|group08to11AM|
|            36|               588395|group12to03PM|
|            38|               462756|group12to03PM|
|            38|               346518|group08to11AM|
|            37|               337074|group12to03PM|
|            14|               274288|group08to11AM|
|            14|               256302|group12to03PM|
|            46|               229325|group12to03PM|
|            20|               219182|group12to03PM|
|            46|               213696|group08to11AM|
|            38|               203232|group04to07PM|
|            71|               201379|group12to03PM|
|            71|               192307|group08to11AM|
|            20|               175688|group08to11AM|
|            21|               148008|group12t

### The results are as below. 
There are 6 bins and their top violation codes are as below. 

group12to03AM -- Violation Codes are 21, 40, 78

group04to07AM -- Violation Codes are 14, 21, 40

group08to11AM -- Violation Codes are 21,36,38

group12to03PM -- Violation Codes are 36,38,37

group04to07PM -- Violation Codes are 38,37,14

group08to11PM -- Violation Codes are 7,38,14



In [28]:
## 5.4 Now, try another direction. For the three most commonly occurring violation codes, find the most common time of the day (in terms of the bins from the previous part).
spark.sql('SELECT `violation code`,count(`violation code`), \
            case \
            when `Violation Hour` IN ("12","01","02","03") and `Violation AMPM` = "A" THEN "group12to03AM" \
            when `Violation Hour` IN ("04","05","06","07") and `Violation AMPM` = "A" THEN "group04to07AM" \
            when `Violation Hour` IN ("08","09","10","11") and `Violation AMPM` = "A" THEN "group08to11AM" \
            when `Violation Hour` IN ("12","01","02","03") and `Violation AMPM` = "P" THEN "group12to03PM" \
            when `Violation Hour` IN ("04","05","06","07") and `Violation AMPM` = "P" THEN "group04to07PM" \
            when `Violation Hour` IN ("08","09","10","11") and `Violation AMPM` = "P" THEN "group08to11PM" \
            END AS time_bin \
            FROM NYC_dfTable \
            where `violation code` in (21,36,38) \
            group by `violation code`,time_bin\
            order by count(`violation code`) desc').show(100)

+--------------+---------------------+-------------+
|violation code|count(violation code)|     time_bin|
+--------------+---------------------+-------------+
|            21|              1182689|group08to11AM|
|            36|               751422|group08to11AM|
|            36|               588395|group12to03PM|
|            38|               462756|group12to03PM|
|            38|               346518|group08to11AM|
|            38|               203232|group04to07PM|
|            21|               148008|group12to03PM|
|            21|               119469|group04to07AM|
|            21|                53600|group12to03AM|
|            38|                47029|group08to11PM|
|            36|                33939|group04to07AM|
|            36|                26858|group04to07PM|
|            21|                23908|         null|
|            38|                 2300|group04to07AM|
|            21|                  551|group04to07PM|
|            38|                  452|group12t

## The results
From the 5.3 we know top occuring codes are 21,36,38 and the time bins which they occur the most(In Descending order) are as below. 

For code 21: group08to11AM,group12to03PM, group04to07AM

For Code 36: group08to11AM, group12to03PM, group04to07AM

For Code 38: group12to03PM, group08to11AM, group04to07PM

In [29]:
## 6.1 First, divide the year into a certain number of seasons, and find frequencies of tickets for each season.
spark.sql('SELECT count(`violation code`) as Winter_Violations\
            FROM NYC_dfTable \
            where Month(`Issue Date`) in (01,02,12)').show()
spark.sql('SELECT count(`violation code`) as Spring_Violations\
            FROM NYC_dfTable \
            where Month(`Issue Date`) in (03,04,05)').show()
spark.sql('SELECT count(`violation code`) as Summer_Violations\
            FROM NYC_dfTable \
            where Month(`Issue Date`) in (06,07,08)').show()
spark.sql('SELECT count(`violation code`) as Fall_Violations\
            FROM NYC_dfTable \
            where Month(`Issue Date`) in (09,10,11)').show()


+-----------------+
|Winter_Violations|
+-----------------+
|          2485331|
+-----------------+

+-----------------+
|Spring_Violations|
+-----------------+
|          2880687|
+-----------------+

+-----------------+
|Summer_Violations|
+-----------------+
|          2606208|
+-----------------+

+---------------+
|Fall_Violations|
+---------------+
|        2830802|
+---------------+



## The Result
We see that the frequency of violations per season as below. 

Winter(Jan, Feb, December):2485331

Spring(March,April,May):2880687

Summner(June,July,August):2606208

Fall(September,October,November):2830802

In [30]:
## 6.2 Then, find the three most common violations for each of these seasons.

spark.sql('SELECT `violation code` Winter_TOP,count(`violation code`)\
            FROM NYC_dfTable \
            where Month(`Issue Date`) in (01,02,12) \
            group by `violation code` \
            order by count(`violation code`) desc limit 3').show()
spark.sql('SELECT `violation code` Spring_TOP,count(`violation code`)\
            FROM NYC_dfTable \
            where Month(`Issue Date`) in (03,04,05) \
            group by `violation code` \
            order by count(`violation code`) desc limit 3').show()
spark.sql('SELECT `violation code` Summer_TOP,count(`violation code`)\
            FROM NYC_dfTable \
            where Month(`Issue Date`) in (06,07,08) \
            group by `violation code` \
            order by count(`violation code`) desc limit 3').show()
spark.sql('SELECT `violation code` FAll_TOP,count(`violation code`)\
            FROM NYC_dfTable \
            where Month(`Issue Date`) in (09,10,11) \
            group by `violation code` \
            order by count(`violation code`) desc limit 3').show()


+----------+---------------------+
|Winter_TOP|count(violation code)|
+----------+---------------------+
|        21|               362341|
|        36|               359338|
|        38|               259723|
+----------+---------------------+

+----------+---------------------+
|Spring_TOP|count(violation code)|
+----------+---------------------+
|        21|               402807|
|        36|               344834|
|        38|               271192|
+----------+---------------------+

+----------+---------------------+
|Summer_TOP|count(violation code)|
+----------+---------------------+
|        21|               405961|
|        38|               247561|
|        36|               240396|
+----------+---------------------+

+--------+---------------------+
|FAll_TOP|count(violation code)|
+--------+---------------------+
|      36|               456046|
|      21|               357479|
|      38|               283828|
+--------+---------------------+



## The Top violation codes as per Season are below.

Winter: Code 21 with 362341 violations

        Code 36 with 359338 violations
        
        Code 38 with 259723 violations
    
Spring: Code 21 with 402807 violations

        Code 36 with 344834 violations
        
        Code 38 with 271192 violations
    
Sumer: Code 21 with 405961 violations
       
       Code 38 with 247561 violations
       
       Code 36 with 240396 violations
    
Fall: Code 36 with 456046 violations

      Code 21 with 357479 violations
      
      Code 38 with 357479 violations  

In [31]:
##The fines collected from all the instances of parking violation constitute a source of revenue for the NYC Police Department. Let’s take an example of estimating this for the three most commonly occurring codes:

## 7.1 Find the total occurrences of the three most common violation codes.
spark.sql('SELECT `violation code` Top_3_Violations,count(`violation code`)\
            FROM NYC_dfTable \
            group by `violation code` \
            order by count(`violation code`) desc limit 3').show()

+----------------+---------------------+
|Top_3_Violations|count(violation code)|
+----------------+---------------------+
|              21|              1528588|
|              36|              1400614|
|              38|              1062304|
+----------------+---------------------+



## The Result

The top 2 violations and thier count is as below. 

Violation code 21 is issued total of 1528588 times.

Violation code 36 is issued total of 1400614 times. 

Violation code 38 is issues total of 1062304 times. 


##7.2 Then, visit the website:http://www1.nyc.gov/site/finance/vehicles/services-violation-codes.page
##It lists the fines associated with different violation codes. They’re divided into two categories: one for the highest-density locations in the city and the other for the rest of the city. For the sake of simplicity, take the average of the two.

The avg fine for the top 3 violations codes ie 21,36,38 are as below.

For violation code 21: 55$
    
For violation code 36: 50$
    
For violation code 38: 50$



In [32]:
##7.3 Using this information, find the total amount collected for the three violation codes with the maximum tickets. State the code that has the highest total collection.

## Total Amount collected with all these 3 ticket codes
spark.sql('SELECT count(`violation code`)*55 as Total_Fine_Via_Code_21 \
            FROM NYC_dfTable \
            where `violation code` = 21 \
            group by `violation code`').show()

spark.sql('SELECT count(`violation code`)*50 as Total_Fine_Via_Code_36 \
            FROM NYC_dfTable \
            where `violation code` = 36 \
            group by `violation code`').show()

spark.sql('SELECT count(`violation code`)*50 as Total_Fine_Via_Code_38 \
            FROM NYC_dfTable \
            where `violation code` = 38 \
            group by `violation code`').show()

+----------------------+
|Total_Fine_Via_Code_21|
+----------------------+
|              84072340|
+----------------------+

+----------------------+
|Total_Fine_Via_Code_36|
+----------------------+
|              70030700|
+----------------------+

+----------------------+
|Total_Fine_Via_Code_38|
+----------------------+
|              53115200|
+----------------------+



In [33]:
spark.sql('SELECT `Registration state`,`violation code`,count(`violation code`) \
           FROM NYC_dfTable \
           where `violation code` in (21,36,38) \
           group by `violation code`,`Registration state`\
           order by count(`violation code`) desc limit 10').show()

+------------------+--------------+---------------------+
|Registration state|violation code|count(violation code)|
+------------------+--------------+---------------------+
|                NY|            36|              1201013|
|                NY|            21|              1145775|
|                NY|            38|               842424|
|                NJ|            21|               105288|
|                NJ|            38|                87051|
|                NJ|            36|                71150|
|                PA|            21|                59124|
|                PA|            36|                34466|
|                FL|            21|                31418|
|                PA|            38|                27564|
+------------------+--------------+---------------------+



In [34]:
## From the above we can see that the state which has got the most number of cases is NY with this we can calculate the total 
## Fine collected. 
spark.sql('SELECT count(`violation code`)*55 as NY_Fine_Via_Code_21 \
            FROM NYC_dfTable \
            where `violation code` = 21 and `Registration state` = "NY" \
            group by `violation code`').show()

spark.sql('SELECT count(`violation code`)*55 as NY_Fine_Via_Code_36 \
            FROM NYC_dfTable \
            where `violation code` = 36 and `Registration state` = "NY"\
            group by `violation code`').show()

spark.sql('SELECT count(`violation code`)*55 as NY_Fine_Via_Code_38 \
            FROM NYC_dfTable \
            where `violation code` = 38 and `Registration state` = "NY" \
            group by `violation code`').show()


+-------------------+
|NY_Fine_Via_Code_21|
+-------------------+
|           63017625|
+-------------------+

+-------------------+
|NY_Fine_Via_Code_36|
+-------------------+
|           66055715|
+-------------------+

+-------------------+
|NY_Fine_Via_Code_38|
+-------------------+
|           46333320|
+-------------------+



## The result: 

The total fine collcted is:63017625+ 66055715 + 46333320 = 175,406,660‬ USD.
    


## 7.3 What can you intuitively infer from these findings?

We can infer that most of the parking violations are done by the New York people, and 

Code 21 "parking in no parking zone"

Code 36 "which is Exceeding the posted speed limit in or near a designated school zone"

Code 38 "Failing to show a receipt or tag in the windshield.Drivers get a 5-minute grace period past the expired time on parking meter receipts".